In [26]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import time
import re

def parse_stars(text):
    if not text:
        return 0
    t = text.strip().replace(',', '').lower()
    m = re.match(r'^([\d\.]+)k$', t)
    if m:
        return int(float(m.group(1)) * 1000)
    try:
        return int(float(t))
    except Exception:
        return 0

db_name = 'google_repos.db'
conn = sqlite3.connect(db_name)
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS repos (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT, language TEXT, stars INTEGER)')
conn.commit()

headers = {"User-Agent": "Mozilla/5.0"}
base_url = "https://github.com/orgs/google/repositories"
all_rows = []

# ページネーション（必要に応じて max_pages を増やす）
max_pages = 1400
for page in range(1, max_pages + 1):
    page_url = f"{base_url}?page={page}"
    print("取得:", page_url)
    time.sleep(0.1)
    try:
        res = requests.get(page_url, headers=headers, timeout=10)
    except requests.RequestException as e:
        print("リクエストエラー:", e)
        break
    if res.status_code != 200:
        print("ステータスコード:", res.status_code)
        break

    soup = BeautifulSoup(res.text, 'html.parser')

    # 候補セレクタを順に試す
    repo_items = soup.select('li.Box-row') or soup.select('li[itemprop="owns"]') or soup.select('div.py-4')
    print("このページの候補数:", len(repo_items))
    if not repo_items:
        break

    for repo in repo_items:
        # name: hrefからも取得可能にする
        name_tag = repo.select_one("a[href^='/google/']") or repo.select_one("a[itemprop='name codeRepository']") or repo.select_one("h3 a")
        if name_tag:
            name = name_tag.get_text(strip=True) or name_tag.get('href', '').split('/')[-1]
        else:
            name = "Unknown"

        # language
        lang_tag = repo.select_one("span[itemprop='programmingLanguage']") or repo.select_one("span[class*='language']") 
        language = lang_tag.get_text(strip=True) if lang_tag else None

        # stars
        star_tag = repo.select_one("a[href$='/stargazers']") or repo.find('a', href=lambda x: x and x.endswith('/stargazers'))
        star_text = star_tag.get_text(strip=True) if star_tag else "0"
        stars = parse_stars(star_text)

        all_rows.append((name, language, stars))

# 一括挿入
if all_rows:
    try:
        cur.executemany('INSERT INTO repos (name, language, stars) VALUES (?, ?, ?)', all_rows)
        conn.commit()
        print(f"{len(all_rows)} 件を保存しました。")
    except sqlite3.Error as e:
        print("DB保存エラー:", e)
else:
    print("取得データなし")

# DB参照
for row in cur.execute('SELECT * FROM repos ORDER BY stars DESC'):
    print(row)

conn.close()

取得: https://github.com/orgs/google/repositories?page=1
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=2
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=3
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=4
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=5
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=6
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=7
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=8
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=9
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=10
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=11
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=12
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=13
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=14
このページの候補数: 2
取得: https://github.com/orgs/google/reposito

In [28]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import time
import re

def parse_stars(text):
    if not text:
        return 0
    t = text.strip().replace(',', '').lower()
    m = re.match(r'^([\d\.]+)k$', t)
    if m:
        return int(float(m.group(1)) * 1000)
    # テキスト中の最初の数値を抽出
    m2 = re.search(r'[\d,\.]+', t)
    if m2:
        try:
            return int(float(m2.group(0).replace(',', '')))
        except Exception:
            return 0
    return 0

DB = 'google_repos.db'
conn = sqlite3.connect(DB)
cur = conn.cursor()
cur.execute('''
CREATE TABLE IF NOT EXISTS repos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE,
    language TEXT,
    stars INTEGER
)
''')
conn.commit()

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
}
base_url = "https://github.com/orgs/google/repositories"
all_rows = []

# ページごとに time.sleep(1) を入れて GitHub に負荷をかけない
max_pages = 1400  # 必要に応じて増減
for page in range(1, max_pages + 1):
    page_url = f"{base_url}?page={page}"
    print("取得:", page_url)
    try:
        res = requests.get(page_url, headers=headers, timeout=15)
    except requests.RequestException as e:
        print("リクエストエラー:", e)
        break

    if res.status_code != 200:
        print("ステータスコード:", res.status_code)
        break

    soup = BeautifulSoup(res.text, 'html.parser')

    # 複数の候補セレクタを試す（GitHub UIにより変わる）
    repo_items = soup.select('li.Box-row') or soup.select('li[itemprop="owns"]') or soup.select('div.py-4') or soup.select('div.col-12.d-flex')
    print("このページの候補数:", len(repo_items))
    if not repo_items:
        break

    for repo in repo_items:
        # 名前
        name_tag = repo.select_one("a[href^='/google/']") or repo.select_one("a[itemprop='name codeRepository']") or repo.select_one("h3 a")
        if name_tag:
            name = name_tag.get_text(strip=True) or name_tag.get('href', '').split('/')[-1]
        else:
            # hrefから抽出できる場合
            href = repo.find('a', href=re.compile(r'/google/[^/]+'))
            name = href['href'].split('/')[-1] if href and href.get('href') else "Unknown"

        # 言語
        lang_tag = repo.select_one("span[itemprop='programmingLanguage']") or repo.select_one("span.color-text-secondary .ml-0, span[class*='language']")
        language = lang_tag.get_text(strip=True) if lang_tag else None

        # スター数（リンクのテキストまたは aria-label を参照）
        star_tag = repo.find('a', href=re.compile(r'/stargazers$')) or repo.select_one('a[href$="/stargazers"]')
        star_text = ""
        if star_tag:
            star_text = star_tag.get_text(strip=True)
            if not star_text:
                star_text = star_tag.get('aria-label', '')  # 例: "123 users starred this repository"
        stars = parse_stars(star_text)

        all_rows.append((name, language, stars))

    # ページごとにスリープ（必須）
    time.sleep(0.01)

if not all_rows:
    print("取得データなし")
else:
    try:
        # name に UNIQUE 制約があるので重複は上書き
        cur.executemany('INSERT OR REPLACE INTO repos (name, language, stars) VALUES (?, ?, ?)', all_rows)
        conn.commit()
        print(f"{len(all_rows)} 件を保存しました。")
    except sqlite3.Error as e:
        print("DB保存エラー:", e)

# 保存データを表示
print("\n-- 保存データ（stars DESC） --")
for row in cur.execute('SELECT name, language, stars FROM repos ORDER BY stars DESC'):
    print(row)

conn.close()

取得: https://github.com/orgs/google/repositories?page=1
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=2
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=3
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=4
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=5
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=6
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=7
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=8
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=9
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=10
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=11
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=12
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=13
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=14
このページの候補数: 2
取得: https://github.com/orgs/google/reposito

In [29]:
# ...existing code...
import sqlite3
import requests
from bs4 import BeautifulSoup
import time
import re

def parse_stars(text):
    if not text:
        return 0
    t = text.strip().lower().replace(',', '')
    m = re.search(r'([\d\.]+)\s*k', t)  # 1.2k など
    if m:
        return int(float(m.group(1)) * 1000)
    m2 = re.search(r'[\d,]+', t)
    if m2:
        try:
            return int(m2.group(0).replace(',', ''))
        except Exception:
            return 0
    return 0

DB = 'google_repos.db'
conn = sqlite3.connect(DB)
cur = conn.cursor()
cur.execute('''
CREATE TABLE IF NOT EXISTS repos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE,
    language TEXT,
    stars INTEGER
)
''')
conn.commit()

headers = {"User-Agent": "Mozilla/5.0"}
base_url = "https://github.com/orgs/google/repositories"
all_rows = []

max_pages = 1500  # 必要に応じて増やす（time.sleep(1) を守ること）
for page in range(1, max_pages + 1):
    page_url = f"{base_url}?page={page}"
    print("取得:", page_url)
    time.sleep(0.01)  # 必ず1秒入れる
    try:
        res = requests.get(page_url, headers=headers, timeout=15)
    except requests.RequestException as e:
        print("リクエストエラー:", e)
        break
    if res.status_code != 200:
        print("ステータスコード:", res.status_code)
        break

    soup = BeautifulSoup(res.text, 'html.parser')

    # 複数候補のセレクタを試す
    repo_items = soup.select('li.Box-row') \
                 or soup.select('li[itemprop="owns"]') \
                 or soup.select('div.py-4') \
                 or soup.select('div.Box-row') \
                 or soup.select('div.col-12.d-flex')
    print("このページの候補数:", len(repo_items))
    if not repo_items:
        # ページにリポジトリ要素が無ければ終了
        break

    for repo in repo_items:
        # 名前： href から確実に抽出
        name = None
        a = repo.select_one("a[href^='/google/']")
        if a and a.get('href'):
            name = a['href'].rstrip('/').split('/')[-1]
        else:
            # 別の構造のフォールバック
            a2 = repo.select_one("h3 a") or repo.select_one("a[itemprop='name codeRepository']")
            if a2:
                name = a2.get_text(strip=True) or (a2.get('href') or '').rstrip('/').split('/')[-1]
        if not name:
            name = "Unknown"

        # 言語
        lang_tag = repo.select_one("span[itemprop='programmingLanguage']") \
                   or repo.select_one("span.color-text-secondary") \
                   or repo.select_one("span[class*='language']")
        language = lang_tag.get_text(strip=True) if lang_tag else None

        # スター数： stargazers リンクまたは aria-label から抽出
        star_tag = repo.find('a', href=re.compile(r'/stargazers$')) or repo.select_one('a[href$="/stargazers"]')
        star_text = ""
        if star_tag:
            star_text = star_tag.get_text(strip=True) or star_tag.get('aria-label', '')
        else:
            # ページ構造によっては svg の隣に数値がある場合もある
            nearby = repo.select_one('svg[aria-label="star"]')
            if nearby:
                parent = nearby.parent
                if parent:
                    star_text = parent.get_text(strip=True)

        stars = parse_stars(star_text)
        all_rows.append((name, language, stars))

# DBへ一括挿入（重複は上書き）
if all_rows:
    try:
        cur.executemany('INSERT OR REPLACE INTO repos (name, language, stars) VALUES (?, ?, ?)', all_rows)
        conn.commit()
        print(f"{len(all_rows)} 件を保存しました。")
    except sqlite3.Error as e:
        print("DB保存エラー:", e)
else:
    print("取得データなし")

# 保存データを表示
print("\n-- 保存データ（stars DESC） --")
for row in cur.execute('SELECT name, language, stars FROM repos ORDER BY stars DESC'):
    print(row)

conn.close()
# ...existing code...

取得: https://github.com/orgs/google/repositories?page=1
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=2
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=3
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=4
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=5
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=6
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=7
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=8
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=9
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=10
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=11
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=12
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=13
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=14
このページの候補数: 2
取得: https://github.com/orgs/google/reposito

In [31]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import time
import re
import random

# スター数のテキストから数値に変換する関数 (1.2k -> 1200 など)
def parse_stars(text):
    """スター数のテキストを整数に変換する"""
    if not text:
        return 0
    t = text.strip().lower().replace(',', '')
    
    # "1.2k stargazers" のような形式に対応
    if 'stargazers' in t:
        t = t.replace('stargazers', '').strip()

    m = re.search(r'([\d\.]+)\s*k', t)  # 1.2k
    if m:
        try:
            return int(float(m.group(1)) * 1000)
        except ValueError:
            return 0
    
    # 単純な数値に対応 (例: 1200, 1,200)
    m2 = re.search(r'[\d]+', t.replace('.', '')) # ドットを除去してから数値のみを抽出
    if m2:
        try:
            return int(m2.group(0))
        except ValueError:
            return 0
            
    return 0

# --- DB初期設定 ---
DB = 'google_repos.db'
conn = sqlite3.connect(DB)
cur = conn.cursor()
cur.execute('''
CREATE TABLE IF NOT EXISTS repos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE,
    language TEXT,
    stars INTEGER
)
''')
conn.commit()

# --- スクレイピング設定 ---
# GitHubはスクレイピングを厳しく監視しているため、User-Agentは必須
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
base_url = "https://github.com/orgs/google/repositories"
all_rows = []

max_pages = 1500  # まずは5ページでテスト。安定したら増やしてください。

# --- スクレイピング実行 ---
for page in range(1, max_pages + 1):
    page_url = f"{base_url}?page={page}"
    print(f"--- ページ {page} 取得開始: {page_url} ---")
    
    # サーバー負荷軽減のため、最低1秒は待機する
    # 念のため、1.0秒から2.0秒のランダムな遅延を入れることで、機械的なアクセスに見えにくくする
    time.sleep(0.01)
    
    try:
        res = requests.get(page_url, headers=headers, timeout=15)
    except requests.RequestException as e:
        print("リクエストエラーが発生しました:", e)
        break
        
    if res.status_code != 200:
        print(f"ステータスコードエラーが発生しました: {res.status_code}")
        # 404 (ページ終了) 以外のエラーであれば、詳細情報を出力して終了
        if res.status_code != 404:
             print("レスポンスの先頭500文字を出力:")
             print(res.text[:500])
        break

    soup = BeautifulSoup(res.text, 'html.parser')

    # リポジトリリストの項目全体を指す、最も安定したセレクタを使用
    # GitHubのリポジトリ一覧は通常、このクラス名を持つ`div`要素で区切られています
    repo_items = soup.select('div.Box-row')
    
    print("このページの候補数:", len(repo_items))

    if not repo_items:
        # リポジトリ要素が見つからなければ、ページネーションの終了と判断
        print("リポジトリが見つかりませんでした。スクレイピングを終了します。")
        break

    for repo in repo_items:
        name = None
        language = None
        stars = 0

        # 1. 名前: リポジトリ名へのリンク要素から抽出
        # hrefが '/google/' で始まるリンクを探す
        name_link = repo.select_one("a[data-testid='repository-name-link']") or repo.select_one("a[href^='/google/']")
        if name_link and name_link.get('href'):
            # URLパスの最後尾（リポジトリ名）を取得
            name = name_link['href'].split('/')[-1].strip()
        else:
            # 別の構造のフォールバック (h3内のaタグなど)
            name_link_fallback = repo.select_one("h3 a")
            if name_link_fallback:
                name = name_link_fallback.get_text(strip=True)

        # 2. 言語: itemprop='programmingLanguage' を持つ span 要素から抽出
        lang_tag = repo.select_one("span[itemprop='programmingLanguage']")
        if lang_tag:
            language = lang_tag.get_text(strip=True)
        
        # 3. スター数: /stargazers へのリンク、またはその aria-label 属性から抽出
        star_link = repo.select_one('a[href$="/stargazers"]') or repo.select_one('a[href*="/stargazers"]')
        star_text = ""
        
        if star_link:
            # 優先度1: アクセシビリティ用のaria-label (例: "12,345 stargazers")
            star_text = star_link.get('aria-label', '')
            
            # 優先度2: 表示されているテキスト (例: "12.3k" または "12,345")
            if not star_text:
                 star_text = star_link.get_text(strip=True)

        stars = parse_stars(star_text)

        if name and stars >= 0:
            all_rows.append((name, language, stars))
            print(f" -> OK: {name} (Lang: {language or 'N/A'}, Stars: {stars})")
        else:
            print(f" -> SKIP: データ不完全 (Name: {name}, Stars Text: {star_text})")


# --- DBへ一括挿入 ---
if all_rows:
    try:
        # INSERT OR REPLACE を使用し、既存のデータは新しい情報で更新（上書き）
        cur.executemany('INSERT OR REPLACE INTO repos (name, language, stars) VALUES (?, ?, ?)', all_rows)
        conn.commit()
        print(f"\n--- 完了: {len(all_rows)} 件のリポジトリ情報をデータベースに保存/更新しました。 ---")
    except sqlite3.Error as e:
        print("DB保存エラー:", e)
else:
    print("\n--- 取得データなし。スクレイピングは成功しませんでした。 ---")

# --- 保存データを表示 ---
print("\n-- データベース内の全データ（stars DESC） --")
count = 0
for row in cur.execute('SELECT name, language, stars FROM repos ORDER BY stars DESC LIMIT 20'):
    print(row)
    count += 1
print(f"（合計 {count} 件を表示）")

conn.close()
message.txt

--- ページ 1 取得開始: https://github.com/orgs/google/repositories?page=1 ---
このページの候補数: 0
リポジトリが見つかりませんでした。スクレイピングを終了します。

--- 取得データなし。スクレイピングは成功しませんでした。 ---

-- データベース内の全データ（stars DESC） --
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
('Unknown', None, 0)
（合計 20 件を表示）


NameError: name 'message' is not defined

In [32]:
import requests
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0"}
url = "https://github.com/orgs/google/repositories?page=1"

res = requests.get(url, headers=headers, timeout=15)
soup = BeautifulSoup(res.text, 'html.parser')

# HTMLの先頭1000文字を出力
print("=== HTML先頭 ===")
print(soup.prettify()[:2000])

# リポジトリ行の候補をすべて探す
print("\n=== 全セレクタの試行結果 ===")
print(f"li.Box-row: {len(soup.select('li.Box-row'))}")
print(f"div.Box-row: {len(soup.select('div.Box-row'))}")
print(f"li[itemprop='owns']: {len(soup.select('li[itemprop=\"owns\"]'))}")
print(f"div.py-4: {len(soup.select('div.py-4'))}")
print(f"*[data-testid='repository-name-link']: {len(soup.select('[data-testid=\"repository-name-link\"]'))}")

# すべてのリンクを表示
print("\n=== /google/で始まるリンク（最初の5件） ===")
links = soup.select("a[href^='/google/']")[:5]
for link in links:
    print(link)

=== HTML先頭 ===
<!DOCTYPE html>
<html data-a11y-animated-images="system" data-a11y-link-underlines="true" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
 <head>
  <meta charset="utf-8"/>
  <link href="https://github.githubassets.com" rel="dns-prefetch"/>
  <link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
  <link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
  <link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
  <link href="https://avatars.githubusercontent.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-8e973f836952.css" media="all" rel="stylesheet">
   <link crossorigin="anonymous" href="https://github.githubassets.com/assets/light_high_contrast-34b642d57214.css" media="all" rel="stylesheet">
    <link crossorigin="anonymous" href="https://github